# Import Libraries

In [1]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [3]:
from text_to_sql import LLMConfig
from core import GeneralLLM
from dotenv import load_dotenv
import pandas as pd

In [4]:
load_dotenv()

True

# Constants

In [5]:
MODEL = "gemini-1.5-pro"
PROVIDER = "gemini"

# Import Dataset

In [6]:
dataset = pd.read_csv(f"../files/dataset/dataset_intent.csv")

In [7]:
dataset

,prompt,is_related
0,Which actor has never appeared in any R-rated ...,1
1,Which language shows up the most in our movie ...,1
2,Which customers have rented the same film more...,1
3,Which movies are in the top 10% by total revenue?,1
4,Bahasa apa yang paling sering muncul di daftar...,1
...,...,...
95,Apa tempat paling indah yang pernah kamu kunju...,0
96,"Kalau bisa ngobrol sama hewan, kamu pilih hewa...",0
97,Apa genre film favorit kamu?,0
98,Apa kebiasaan unik yang kamu punya?,0


# Initialize LLM

In [8]:
# Configurations
provider_key = PROVIDER.upper().replace("-", "_")
general_config = LLMConfig(
    type="api",
    model=MODEL,
    provider=PROVIDER,
    api_key=os.getenv(f"API_KEY_{provider_key}"),
)

# Initialize agents
llm_agent = GeneralLLM(config=general_config)

Initializing API client for gemini using model gemini-1.5-pro.


# Define Function

In [13]:
def detect_intent_tool_baseline(query, llm_agent):
    """
    Baseline: Directly ask the model if the query is about data retrieval.
    Return 1 for 'data' intent, 0 for 'other' (no explanation, no reasoning).
    """

    system_prompt = "Classify this query as '1' if it's about retrieving data from a database, or '0' if not."
    user_prompt = f"Query: {query}\nAnswer:"

    result = llm_agent.generate(system_prompt=system_prompt.strip(), user_prompt=user_prompt.strip())
    final = result.strip()

    if final not in ["0", "1"]:
        final = "1"

    return int(final)

In [9]:
def detect_intent_tool(query, llm_agent):
    """
    Detect whether the query is about retrieving data from a database (for SQL generation).
    Return 'data' if it is, otherwise 'other'.
    """

    system_prompt = f"""
    You are an expert assistant that decides if a user query is asking to retrieve data from a database.
    Return ONLY one of the following labels:

    - data: if the query is asking about facts, entities, statistics, filters, aggregations, rankings, comparisons, counts, sums, averages, or any structured information that could be stored in tables
    - other: if the query is not about retrieving data (e.g., definitions, instructions, general advice, or how-to questions)

    THINK STEP BY STEP:
    1. Does the query ask for specific information that could be stored in database tables?
    2. Would answering this require looking up records or performing calculations on stored data?
    3. If yes to either, it's 'data'

    Here are some EXAMPLES:

    Query: Which actors have the first name 'Scarlett'?  
    → data (lookup in actor table)

    Query: How many distinct actor last names are there?  
    → data (count operation)

    Query: What is SQL?  
    → other (definition)

    Query: What does SELECT do in SQL?  
    → other (explanation)

    Query: Which customers rented more than 3 categories?  
    → data (filtering and counting)

    Query: Compare sales between 2022 and 2023  
    → data (comparison of stored data)

    Query: How do I write a JOIN query?  
    → other (instruction)

    Query: What's the average salary by department?  
    → data (aggregation)

    Now, based on the query below, return only `data` or `other` (lowercase only — no explanation).

    Query:
    \"\"\"{query}\"\"\"
    """

    result = llm_agent.generate(system_prompt=system_prompt.strip(), user_prompt="")
    final = result.strip().lower()

    if final not in ["data", "other"]:
        final = "data"

    if final == "data":
        return 1
    else:
        return 0

# Experiment

## Baseline

In [14]:
predictions = []
for _, row in dataset.iterrows():
    prediction = detect_intent_tool_baseline(row["prompt"], llm_agent)
    predictions.append(prediction)

dataset["predicted_is_related_baseline"] = predictions

accuracy = (dataset["predicted_is_related_baseline"] == dataset["is_related"]).mean()
print(f"Baseline Accuracy: {accuracy:.2%}")

Baseline Accuracy: 87.00%


## With Prompt Engineering

In [ ]:
predictions = []
for _, row in dataset.iterrows():
    prediction = detect_intent_tool(row["prompt"], llm_agent)
    predictions.append(prediction)

dataset["predicted_is_related"] = predictions
accuracy = (dataset["predicted_is_related"] == dataset["is_related"]).mean()

print(f"Prompt Engineering Accuracy: {accuracy:.2%}")

Accuracy: 100.00%


# Save Prediction

In [15]:
dataset.to_csv(f"../files/experiment_result/intent_detection/{MODEL}-result.csv", index=False)